In [ ]:
import datetime
from pathlib import Path
from typing import Union

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.backends.backend_pgf import FigureCanvasPgf

matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)
sns.set_theme()
plt.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': '\\usepackage{lmodern}',
})

frames = [
    pd.read_csv(Path.cwd().joinpath('data/vienna_20100101_20131231.csv'), index_col='time', parse_dates=True),
    pd.read_csv(Path.cwd().joinpath('data/vienna_20140101_20171231.csv'), index_col='time', parse_dates=True),
    pd.read_csv(Path.cwd().joinpath('data/vienna_20180101_20211231.csv'), index_col='time', parse_dates=True)
]
df = pd.concat(frames)
df.index = df.index.tz_convert(None)
df

In [ ]:
def get_unique_column_values(df: pd.DataFrame) -> [str]:
    # returns a list of all columns in the dataframe that contain only one unique value (i.e. all rows are equal)
    # cf. https://stackoverflow.com/a/54405767
    def is_unique(s: pd.Series):
        a = s.to_numpy()
        return (a[0] == a).all()

    result = []
    for col in df.columns:
        if is_unique(df[col]):
            print(f'Column {col} has only a single value: {df[col][0]}')
            result.append(col)

    return result


def remove_duplicate_indices(df: Union[pd.DataFrame, pd.Series]) -> Union[pd.DataFrame, pd.Series]:
    duplicates = df[df.index.duplicated(keep=False)]
    if duplicates.empty:
        print('There are no duplicate indices')
        return df
    print('Duplicated indices:')
    print(duplicates.index)

    remove = df.index.duplicated(keep='last')
    return df[~remove]


# Remove columns without any information and duplicate indices
df.drop(columns=get_unique_column_values(df), inplace=True)
df = remove_duplicate_indices(df)
df

We will predict the air temperature measurements 2m above ground ("TL").
Possible input attributes are air temperature (TL), air pressure (P), reduced air pressure (P0), wind direction (DD), mean wind speed (FFAM), relative humidity (RF), precipitation (RR), sun shine duration (SO), and dew point (TP).

In [ ]:
# first, we check whether the index is indeed complete (every 10 minutes)
print(f'Dataset ranging from {df.index.min()} to {df.index.max()} in 10-minute steps:')
(df.index == pd.date_range(df.index.min(), df.index.max(), freq='10min')).all(axis=0)

In [ ]:
# let's see how complete the data is
df[df.isna().any(axis=1)]

In [ ]:
df['TL'].plot()
plt.show()

In [ ]:
# we have many gaps due to incomplete reduced air pressure (P0)
# if we remove the column, we halve the number of measurements with missing values
without_p0 = df.drop(columns=['P0', 'P0_FLAG'])
df[without_p0.isna().any(axis=1)]

In [ ]:
# another suspect of many missing values is the dew point (TP)
# then we suddenly only remain with 0.3% missing values
df = without_p0
without_tp = df.drop(columns=['TP', 'TP_FLAG'])
df[without_tp.isna().any(axis=1)]

In [ ]:
# for small gaps (up to 3 hours) we use simple linear interpolation
df = without_tp
df = df.interpolate(method='linear', limit=17, limit_area='inside')
gaps = df[df.isna().any(axis=1)]
gaps

In [ ]:
# we still have some bigger gaps in the data concentrated on a few days
# interpolation is not sufficient, as we cannot interpolate over a gap of multiple days.
gap_days = gaps.index.map(pd.Timestamp.date).unique()
print('Missing values on :')
for day in gap_days:
    daily = df.loc[str(day)]
    missing = daily[daily.isna().any(axis=1)]
    print(f'{day}: {len(missing)}\t(={len(missing) / (60 / 10 * 24) * 100:.2f}%)')

In [ ]:
# let's look at the gaps one after another
from datetime import datetime, timedelta


def extend_gap(gap: slice, delta: timedelta):
    return slice(datetime.fromisoformat(gap.start) - delta, datetime.fromisoformat(gap.stop) + delta)


# the first one has only missing precipitation
gap = slice('2010-07-29 15:40:00', '2010-07-30 09:50:00')
df.loc[extend_gap(gap, timedelta(hours=4)), 'RR'].plot()
plt.show()
# it is reasonable to let it stop raining
df.loc[gap, 'RR'] = 0

In [ ]:
# next is again a precipitation gap
gap = slice('2011-10-07 13:30:00', '2011-10-08 00:00:00')
df.loc[extend_gap(gap, timedelta(hours=6)), ['RR']].plot()
plt.show()
# again, we can safely assume it stopped raining
df.loc[gap, ['RR']] = 0

In [ ]:
# another precipitation gap
gap = slice('2011-10-09 09:20:00', '2011-10-09 12:10:00')
df.loc[extend_gap(gap, timedelta(hours=6)), ['RR']].plot()
plt.show()
# probably hasn't rained
df.loc[gap, ['RR']] = 0

In [ ]:
# the next one is a bigger wind direction + speed gap
gap = slice('2014-11-29 11:10:00', '2014-12-01 10:40:00')
df.loc[extend_gap(gap, timedelta(hours=72)), ['DD', 'FFAM']].plot()
plt.show()
# let's just re-use the last 24 hours over the period
for dt in pd.date_range(gap.start, gap.stop, freq='10min'):
    df.loc[dt, ['DD', 'FFAM']] = df.loc[dt - timedelta(hours=24), ['DD', 'FFAM']]

In [ ]:
gap = slice('2017-04-02 02:00:00', '2017-04-02 05:00:00')
df.loc[extend_gap(gap, timedelta(hours=24)), ['DD', 'FFAM', 'P']].plot()
plt.show()
# linear interpolation should be fine also here
df.loc[extend_gap(gap, timedelta(hours=1))] = df.loc[extend_gap(gap, timedelta(hours=1))].interpolate(method='linear',
                                                                                                      limit=10,
                                                                                                      limit_area='inside')

In [ ]:
gap = slice('2017-04-04 07:40:00', '2017-04-04 14:10:00')
df.loc[extend_gap(gap, timedelta(hours=48)), ['DD', 'FFAM']].plot()
plt.show()
# wind has not changed significantly, let's repeat the past 4 hours
for dt in pd.date_range(gap.start, gap.stop, freq='10min'):
    df.loc[dt, ['DD', 'FFAM']] = df.loc[dt - timedelta(hours=4), ['DD', 'FFAM']]

In [ ]:
# now we have the biggest gap in the data
# %matplotlib qt
gap = slice('2017-04-05 23:50:00', '2017-04-09 23:50:00')
df.loc[extend_gap(gap, timedelta(hours=144))].plot()
plt.show()
# we cannot identify a significant weather change in these 4 days, hence we repeat the last 4 days
# only the wind direction has shifted a bit, but we would not know how to represent this in the data
for dt in pd.date_range(gap.start, gap.stop, freq='10min'):
    past_hours = [24, 48, 72]
    df.loc[dt] = 0
    for h in past_hours:
        df.loc[dt] += df.loc[dt - timedelta(h)]
    df.loc[dt] /= len(past_hours)
# %matplotlib inline

In [ ]:
gap = slice('2018-04-01 06:10:00', '2018-04-03 05:40:00')
df.loc[extend_gap(gap, timedelta(hours=144)), ['DD', 'FFAM']].plot()
plt.show()
# three days ago a similar pattern occurred, we fill it up
for dt in pd.date_range(gap.start, gap.stop, freq='10min'):
    past_hours = [24, 48, 72]
    df.loc[dt] = 0
    for h in past_hours:
        df.loc[dt] += df.loc[dt - timedelta(h)]
    df.loc[dt] /= len(past_hours)

In [ ]:
# and finally the COVID gap
gap = slice('2020-03-13 20:10:00', '2020-03-15 12:40:00')
df.loc[extend_gap(gap, timedelta(hours=144))].plot()
plt.show()
# we take the averages of the past three days
for dt in pd.date_range(gap.start, gap.stop, freq='10min'):
    past_hours = [24, 48, 72]
    df.loc[dt] = 0
    for h in past_hours:
        df.loc[dt] += df.loc[dt - timedelta(h)]
    df.loc[dt] /= len(past_hours)

In [ ]:
# do a simple plausibility check of the final data (flag values above 300 indicate a potential faulty measurement)
df[(df['TL'] < -15) | (df['TL'] > 40) | (df['TL_FLAG'] > 300) | (df['RF_FLAG'] > 300) | (df['P_FLAG'] > 300)]

In [ ]:
# remove the remaining flag attributes and arrive at a dataset without null values
df.drop(columns=['DD_FLAG', 'FFAM_FLAG', 'P_FLAG', 'RF_FLAG', 'RR_FLAG', 'SO_FLAG', 'TL_FLAG'], inplace=True)
print(f'Remaining NaN values: {df[df.isna().any(axis=1)]}')
df.describe()

In [ ]:
# we need to encode the degrees of the wind direction: 360° should be close to 0°
df.loc[:, 'DD_sin'] = np.sin(df.loc[:, 'DD'] * np.pi / 180)
df.loc[:, 'DD_cos'] = np.cos(df.loc[:, 'DD'] * np.pi / 180)

In [ ]:
# in the correlation matrix we see that all attributes are quite unique
df.corr()

In [ ]:
df.to_pickle(Path.cwd().joinpath('zamg_vienna.pickle'))
df

In [ ]:
# when we sample hourly data we need to sum up the sunshine duration and precipitation for 1 hour
df['SO'] = df['SO'].rolling(6).sum()
df['RR'] = df['RR'].rolling(6).sum()
df = df.iloc[6:, :]  # remove created NaN entries (start with next full hour)
df

In [ ]:
# be aware that precipitation is a very skewed distribution
# most of the time it is not raining, but sometimes it rains a lot
print(df['RR'].describe())
df['RR'].plot.box()
plt.show()

In [ ]:
# we create an additional variant where 'RR' is min-max normalized
# equals a MaxAbsScaler, since the minimum value is 0
df['RR_norm'] = (df.loc[:, 'RR'] - df['RR'].min()) / (df['RR'].max() - df['RR'].min())
print(df['RR_norm'].describe())
df.to_pickle(Path.cwd().joinpath('zamg_vienna_hourly.pickle'))
df